<a href="https://colab.research.google.com/github/PJMS/p-silmu-2021/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_data = pd.read_csv('/content/drive/MyDrive/heart stroke/train_strokes.csv')
raw_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


## 데이터 분석

각 컬럼의 고윳값을 알아봄

In [ ]:
raw_data.nunique()

id                   43400
gender                   3
age                    104
hypertension             2
heart_disease            2
ever_married             2
work_type                5
Residence_type           2
avg_glucose_level    12543
bmi                    555
smoking_status           3
stroke                   2
dtype: int64

In [ ]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43400 entries, 0 to 43399
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 43400 non-null  int64  
 1   gender             43400 non-null  object 
 2   age                43400 non-null  float64
 3   hypertension       43400 non-null  int64  
 4   heart_disease      43400 non-null  int64  
 5   ever_married       43400 non-null  object 
 6   work_type          43400 non-null  object 
 7   Residence_type     43400 non-null  object 
 8   avg_glucose_level  43400 non-null  float64
 9   bmi                41938 non-null  float64
 10  smoking_status     30108 non-null  object 
 11  stroke             43400 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 4.0+ MB


흡연 상태와 bmi수치 컬럼에 결측치가 있는것 확인. 글루코스 수치는 개인이 구하기 힘든 특징이라고 생각해 해당 컬럼을 제거했음

In [ ]:
heart_df = raw_data.drop('avg_glucose_level',axis=1)

In [ ]:
heart_df.head(1)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,18.0,NaN,0


####  데이터들의 고윳값 확인

In [ ]:
print('성별\n',heart_df['gender'].value_counts(),'\n')
print('결혼 여부\n',heart_df['ever_married'].value_counts(),'\n')
print('일의 종류\n',heart_df['work_type'].value_counts(),'\n')
print('주거 장소\n',heart_df['Residence_type'].value_counts(),'\n')
print('심장마비로 사망 여부\n',heart_df['stroke'].value_counts(),'\n')

성별
 Female    25665
Male      17724
Other        11
Name: gender, dtype: int64 

결혼 여부
 Yes    27938
No     15462
Name: ever_married, dtype: int64 

일의 종류
 Private          24834
Self-employed     6793
children          6156
Govt_job          5440
Never_worked       177
Name: work_type, dtype: int64 

주거 장소
 Urban    21756
Rural    21644
Name: Residence_type, dtype: int64 

심장마비로 사망 여부
 0    42617
1      783
Name: stroke, dtype: int64 



#### 성별 분석 후 전처리

In [ ]:
other_c = heart_df['gender'] == 'Other'
is_other = heart_df[other_c]
is_other

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,bmi,smoking_status,stroke
2914,56156,Other,26.0,0,0,No,Private,Rural,22.4,formerly smoked,0
10768,26188,Other,46.0,0,0,No,Private,Rural,NaN,never smoked,0
15620,52984,Other,27.0,0,0,No,Private,Urban,NaN,smokes,0
15838,24893,Other,53.0,0,0,Yes,Private,Rural,NaN,NaN,0
19748,13471,Other,39.0,0,0,Yes,Govt_job,Urban,32.0,formerly smoked,0
21041,31015,Other,19.0,0,0,No,Private,Rural,32.2,formerly smoked,0
21192,27043,Other,49.0,0,0,Yes,Private,Urban,37.2,never smoked,0
24252,51111,Other,51.0,0,0,Yes,Private,Rural,32.2,formerly smoked,0
26199,42452,Other,10.0,0,0,No,children,Rural,21.2,formerly smoked,0
26831,49812,Other,53.0,0,0,Yes,Govt_job,Urban,NaN,NaN,0


성별중에 other값이라는게 있어서 따로 로우를 설정해 확인해본 결과 그 수가 몇개 안되고 stroke값도 전부 0으로 되어있어 결과 예측에 별 도움이 되지 않을거라고 판단해 해당 로우를 제거하기로함

In [ ]:
gender_condition =heart_df[heart_df['gender'] == 'Other'].index
heart_df.drop(gender_condition,inplace=True)

In [ ]:
len(heart_df)

43389

##### 흡연의 여부가 결과에 미치는 영향

In [ ]:
smoke_df=heart_df.groupby('smoking_status')['stroke'].value_counts()

In [ ]:
smoke_df

smoking_status   stroke
formerly smoked  0          7266
                 1           221
never smoked     0         15767
                 1           284
smokes           0          6428
                 1           133
Name: stroke, dtype: int64

흡연 여부가 심장마비로 사망할 확률에 미치는 영향

In [ ]:
for i in range(6):
    if (i%2) == 0:
        percentage = smoke_df[i+1]/(smoke_df[i]+smoke_df[i+1]) * 100
        print(round(percentage,2),"%")

2.95 %
1.77 %
2.03 %


#### bmi수치가 결과에 미치는 영향 확인

In [ ]:
# heart_df.loc[heart_df['bmi'] < 23 , 'bmi_rank']  = '정상'
# heart_df.loc[(heart_df['bmi'] >= 23) & (heart_df['bmi'] <= 24.9) , 'bmi_rank']  = '과체중'
# heart_df.loc[(heart_df['bmi'] > 24.9) & (heart_df['bmi'] <= 29.9) ,'bmi_rank']  = '비만_1'
# heart_df.loc[(heart_df['bmi'] > 29.9) & (heart_df['bmi'] <= 34.9) , 'bmi_rank']  = '비만_2'
# heart_df.loc[heart_df['bmi'] > 34.9 , 'bmi_rank']  = '비만_3'
# heart_df.loc[heart_df['bmi'].isnull() , 'bmi_rank']  = 'NaN'

In [ ]:
heart_df.head(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,bmi,smoking_status,stroke,bmi_rank
0,30669,Male,3.0,0,0,No,children,Rural,18.0,NaN,0,정상
1,30468,Male,58.0,1,0,Yes,Private,Urban,39.2,never smoked,0,비만_3
2,16523,Female,8.0,0,0,No,Private,Urban,17.6,NaN,0,정상


In [ ]:
print(heart_df['bmi_rank'].value_counts())

비만_1    11694
정상       9851
비만_2     8396
비만_3     7615
과체중      4375
NaN      1458
Name: bmi_rank, dtype: int64


In [ ]:
bmi_df=heart_df.groupby('bmi_rank')['stroke'].value_counts()
bmi_df

bmi_rank  stroke
NaN       0          1318
          1           140
과체중       0          4320
          1            55
비만_1      0         11463
          1           231
비만_2      0          8231
          1           165
비만_3      0          7498
          1           117
정상        0          9776
          1            75
Name: stroke, dtype: int64

bmi수치가 심장마비로 인해 사망에 미치는 영향(널값 포함)

In [ ]:
for i in range(12):
    if (i%2) == 0:
        percentage = bmi_df[i+1]/(bmi_df[i]+bmi_df[i+1]) * 100
        print(round(percentage,2),"%")

9.6 %
1.26 %
1.98 %
1.97 %
1.54 %
0.76 %


그래도 정상 범주의 bmi를 가진 사람들이 심장 마비로 사망할 확률이 낮은것을 확인 가능함

bmi의 결측치를 채워넣기 위해 연령&성별 bmi수치를 가져와 각 조건에 맞는 bmi 수치를 입력해줌

In [ ]:
heart_df['age'].describe()

count    43389.000000
mean        42.219747
std         22.520750
min          0.080000
25%         24.000000
50%         44.000000
75%         60.000000
max         82.000000
Name: age, dtype: float64

https://mangome.tistory.com/245 

In [ ]:
condition_1_m = (heart_df['gender'] == 'Male') & (heart_df['age'] <10 ) # 각 연령과 성별에 맞는 조건식 설정
condition_1_f = (heart_df['gender'] == 'Female') & (heart_df['age'] <10 )

condition_2_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=10) & (heart_df['age'] < 20) )
condition_2_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=10) & (heart_df['age'] < 20) )

condition_3_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=20) & (heart_df['age'] < 30) )
condition_3_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=20) & (heart_df['age'] < 30 ))

condition_4_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=30) & (heart_df['age'] < 40) )
condition_4_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=30) & (heart_df['age'] < 40) )

condition_5_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=40) & (heart_df['age'] < 50) )
condition_5_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=40) &( heart_df['age'] < 50 ))

condition_6_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=50) & (heart_df['age'] < 60) )
condition_6_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=50) & (heart_df['age'] < 60) )

condition_7_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=60) & (heart_df['age'] < 70) )
condition_7_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=60) & (heart_df['age'] < 70) )

condition_8_m = (heart_df['gender'] == 'Male') & ((heart_df['age'] >=70) & (heart_df['age'] < 80) )
condition_8_f = (heart_df['gender'] == 'Female') & ((heart_df['age'] >=70) & (heart_df['age'] < 80) )

condition_9_m = (heart_df['gender'] == 'Male') & (heart_df['age'] >=80 )
condition_9_f = (heart_df['gender'] == 'Female') & (heart_df['age'] >=80 )

In [ ]:
heart_df.loc[(condition_1_f)&(heart_df.bmi.isna()),'bmi'] = 19.46
heart_df.loc[(condition_1_m)&(heart_df.bmi.isna()),'bmi'] = 21.01

heart_df.loc[(condition_2_f)&(heart_df.bmi.isna()),'bmi'] = 20.52
heart_df.loc[(condition_2_m)&(heart_df.bmi.isna()),'bmi'] = 22.09

heart_df.loc[(condition_3_f)&(heart_df.bmi.isna()),'bmi'] = 21.20
heart_df.loc[(condition_3_m)&(heart_df.bmi.isna()),'bmi'] = 22.98

heart_df.loc[(condition_4_f)&(heart_df.bmi.isna()),'bmi'] = 21.91
heart_df.loc[(condition_4_m)&(heart_df.bmi.isna()),'bmi'] = 23.74

heart_df.loc[(condition_5_f)&(heart_df.bmi.isna()),'bmi'] = 22.76
heart_df.loc[(condition_5_m)&(heart_df.bmi.isna()),'bmi'] = 24.35

heart_df.loc[(condition_6_f)&(heart_df.bmi.isna()),'bmi'] = 23.46
heart_df.loc[(condition_6_m)&(heart_df.bmi.isna()),'bmi'] = 25.00

heart_df.loc[(condition_7_f)&(heart_df.bmi.isna()),'bmi'] = 24.15
heart_df.loc[(condition_7_m)&(heart_df.bmi.isna()),'bmi'] = 25.68

heart_df.loc[(condition_8_f)&(heart_df.bmi.isna()),'bmi'] = 25.65
heart_df.loc[(condition_8_m)&(heart_df.bmi.isna()),'bmi'] = 26.54

heart_df.loc[(condition_9_f)&(heart_df.bmi.isna()),'bmi'] = 27.16
heart_df.loc[(condition_9_m)&(heart_df.bmi.isna()),'bmi'] = 27.75

In [ ]:
heart_df.isnull().sum()

id                    0
gender                0
age                   0
hypertension          0
heart_disease         0
ever_married          0
work_type             0
Residence_type        0
bmi                   0
smoking_status    13290
stroke                0
dtype: int64

#### 다시 한 번 범위를 설정해줌
널 값이 사라진걸 확인 함

In [ ]:
# heart_df.loc[heart_df['bmi'] < 23 , 'bmi_rank']  = '정상'
# heart_df.loc[(heart_df['bmi'] >= 23) & (heart_df['bmi'] <= 24.9) , 'bmi_rank']  = '과체중'
# heart_df.loc[(heart_df['bmi'] > 24.9) & (heart_df['bmi'] <= 29.9) ,'bmi_rank']  = '비만_1'
# heart_df.loc[(heart_df['bmi'] > 29.9) & (heart_df['bmi'] <= 34.9) , 'bmi_rank']  = '비만_2'
# heart_df.loc[heart_df['bmi'] > 34.9 , 'bmi_rank']  = '비만_3'
# heart_df.loc[heart_df['bmi'].isnull() , 'bmi_rank']  = 'NaN'

In [ ]:
print(heart_df['bmi_rank'].value_counts())

비만_1    12351
정상      10225
비만_2     8396
비만_3     7615
과체중      4802
Name: bmi_rank, dtype: int64


최종적으로 bmi 범위에 따른 심장 마비 확률을 구함

In [ ]:
bmi_df=heart_df.groupby('bmi_rank')['stroke'].value_counts()
bmi_df

bmi_rank  stroke
과체중       0          4722
          1            80
비만_1      0         12015
          1           336
비만_2      0          8231
          1           165
비만_3      0          7498
          1           117
정상        0         10140
          1            85
Name: stroke, dtype: int64

In [ ]:
for i in range(10):
    if (i%2) == 0:
        percentage = bmi_df[i+1]/(bmi_df[i]+bmi_df[i+1]) * 100
        print(round(percentage,2),"%")

1.67 %
2.72 %
1.97 %
1.54 %
0.83 %


#### 흡연 여부의 결측치는 전부 비흡연자로 설정 후 최종 저장

In [ ]:
heart_df['smoking_status'].fillna('never smoked',inplace = True)

In [ ]:
heart_df.isnull().sum()

id                0
gender            0
age               0
hypertension      0
heart_disease     0
ever_married      0
work_type         0
Residence_type    0
bmi               0
smoking_status    0
stroke            0
dtype: int64

In [ ]:
heart_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,18.0,never smoked,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,17.6,never smoked,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,19.1,never smoked,0


bmi범위를 구했으니 부동소수로된 bmi컬럼은 삭제를 한다

In [ ]:
preprocessing_data = heart_df.drop('bmi',axis=1)

In [ ]:
preprocessing_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,never smoked,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,never smoked,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,never smoked,0


In [ ]:
preprocessing_data.to_csv('/content/drive/MyDrive/heart stroke/prepro_data_2.csv')